<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Chat Completion for Decisions</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

----
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.60.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.6 Microsoft.SemanticKernel, 1.60.0

Using Azure OpenAI Service


### Step 2 - Execute Decision Chat Completion with Prompt Execution Settings 

> 📜 **_"If you can’t describe what you are doing as a process, you don’t know what you’re doing."_** 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

// Display the result of the chat completion request as Markdown
chatResult.Content.DisplayAs("text/markdown");

### 1. SWOT Analysis

SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework helps individuals or organizations evaluate internal and external factors that can impact a decision, leading to a more comprehensive understanding of the situation.

  

### 2. Decision Matrix (Weighted Scoring Model)

A decision matrix allows you to compare multiple options against a set of criteria. Each criterion is weighted based on importance, and each option is scored, making it easier to identify the best choice objectively.

  

### 3. Cost-Benefit Analysis

This framework involves listing and quantifying all the costs and benefits associated with each option. By comparing the total expected costs against the total expected benefits, decision-makers can determine which option offers the greatest net benefit.

  

### 4. OODA Loop

OODA stands for Observe, Orient, Decide, and Act. This iterative framework is especially useful in fast-changing environments, encouraging continuous assessment and adaptation throughout the decision-making process.

  

### 5. Six Thinking Hats

Developed by Edward de Bono, this framework encourages looking at a decision from six different perspectives (logic, emotion, caution, optimism, creativity, and process), helping to uncover blind spots and improve the overall quality of decisions.

----
### Step 3 - Execute Decision Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt, openAIPromptExecutionSettings))
{
    Console.Write(content);
}

### 1. SWOT Analysis

SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework helps individuals or organizations evaluate internal and external factors that can impact a decision, leading to a more comprehensive understanding of the situation.



### 2. Decision Matrix (Weighted Scoring Model)

A decision matrix allows you to compare multiple options against a set of criteria. Each criterion is weighted based on importance, and each option is scored, helping to objectively identify the best choice.



### 3. Cost-Benefit Analysis

This framework involves listing all the costs and benefits associated with each option. By quantifying and comparing these, decision-makers can determine which option provides the greatest net benefit.



### 4. The OODA Loop

OODA stands for Observe, Orient, Decide, and Act. Originally developed for military strategy, this iterative framework helps individuals and teams make quick, adaptive decisions in dynamic environments.



### 

----
### Step 4 - Execute Decision Chat Completion with a System Prompt & Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

// Note: The chat completion service can also be used with a chat history object, which allows for a more conversational context.
var chatResponse = string.Empty;

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add the assistant's response to the chat history
chatHistory.AddAssistantMessage(chatResponse);

### 1. Pros and Cons Analysis

**Overview:**  
Making a list of positive and negative aspects of each option.

**How It Helps:**  
- Clarifies trade-offs and consequences  
- Makes implicit biases explicit  
- Supports prioritization and transparent evaluation

**Steps:**  
1. Define the decision clearly.  
2. List the pros of each option.  
3. List the cons of each option.  
4. Weigh the importance of each factor.  
5. Make a balanced decision based on the analysis.

  

### 2. Cost-Benefit Analysis

**Overview:**  
Comparing the expected costs and benefits (quantitative or qualitative) of each possible action.

**How It Helps:**  
- Optimizes resource allocation  
- Allows for direct comparison using a common metric  
- Supports justification and communication of chosen option

**Steps:**  
1. Identify all costs and benefits for each option.  
2. Quantify costs and benefits where possible.  
3. Calculate net value or ratio.  
4. Assess risks and uncertainties.  
5. Choose the option with the best balance of benefits over costs.

  

### 3. Six Thinking Hats (Edward de Bono)

**Overview:**  
A structured framework to look at decisions from multiple perspectives, ensuring holistic thinking.

**How It Helps:**  
- Reduces groupthink and bias  
- Ensures emotional, factual, creative, and critical perspectives are considered

**Steps:**  
1. White Hat: Focus on data and facts.  
2. Red Hat: Explore emotions and feelings.  
3. Black Hat: Identify risks, difficulties, and problems.  
4. Yellow Hat: Look for value and benefits.  
5. Green Hat: Generate creative solutions.  
6. Blue Hat: Manage the decision process.

  

### 4. OODA Loop (Observe, Orient, Decide, Act)

**Overview:**  
A cycle for rapid, iterative decision-making, especially in fast-changing environments.

**How It Helps:**  
- Supports adaptation and agility  
- Encourages continual learning and improvement

**Steps:**  
1. Observe: Gather information and data.  
2. Orient: Analyze information, set context, and generate insights.  
3. Decide: Choose the best course of action.  
4. Act: Execute the chosen action and loop back based on results.

  

### 5. Decision Matrix (Weighted Scoring Model)

**Overview:**  
A quantitative approach to systematically compare multiple options based on multiple criteria.

**How It Helps:**  
- Reduces subjectivity  
- Clarifies which criteria matter most  
- Makes complex decisions more transparent

**Steps:**  
1. List the decision criteria and options.  
2. Assign a weight to each criterion according to importance.  
3. Score each option for every criterion.  
4. Multiply scores by weights and sum for each option.  
5. Select the option with the highest score.

  

Using one or more of these frameworks can improve the structure, rigor, and clarity of your decision-making processes.

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> 📜 **_"Take time to deliberate, but when the time for action comes, stop thinking and go in."_**
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [6]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of the 5 decision frameworks listed above is best suited for the military intelligence community?  
Think carefully step by step about what decision frameworks are needed to answer the query.  
Select only the single best framework. 
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

### Step-by-Step Reasoning

#### 1. Understanding Military Intelligence Decision Needs

- **Nature of Decisions:** The military intelligence community routinely faces situations that are high-stakes, time-sensitive, and rapidly evolving.
- **Key Requirements:**  
  - Rapid information assimilation  
  - Adaptation to changing circumstances  
  - Timely decision cycles  
  - Iterative feedback and adjustment  
  - Managing incomplete or ambiguous data

#### 2. Evaluating Each Framework

**Pros and Cons Analysis:**  
- Good for weighing options, but less suited for high-speed, dynamic environments where continual reassessment is needed.

**Cost-Benefit Analysis:**  
- Useful for longer-term or resource allocation decisions, but not optimized for the rapid tempo and uncertainty typical in military intelligence.

**Six Thinking Hats:**  
- Encourages holistic analysis and team perspectives, but can be too slow and deliberative in time-critical scenarios.

**OODA Loop (Observe, Orient, Decide, Act):**  
- Designed for fast, flexible, and iterative decision-making.  
- Developed by military strategist John Boyd specifically to help with rapid adaptation and competitive advantage in conflict situations.  
- Emphasizes continuous learning, adaptation, and action based on evolving intelligence.  
- Matches the military’s need for agility, feedback loops, and responding to fluid adversary actions.

**Decision Matrix (Weighted Scoring Model):**  
- Great for structured, high-consensus environments with stable criteria; less suited for real-time shifts and ambiguity.

#### 3. Conclusion

- **Most frameworks are valuable, but the military intelligence community above all requires adaptability, speed, and the ability to act under uncertainty and ambiguity.**
- The OODA Loop stands out as specifically designed for these contexts and is widely used in military strategy.

### Best-Suited Framework

**OODA Loop (Observe, Orient, Decide, Act)**  
- It best addresses the unique, high-tempo, and adaptive decision-making needs that define the military intelligence community.

### Step 5 - Execute Decision Chat Completion with a Multi-Turn Conversation Scenario 

> 📜 **_"Price is what you pay. Value is what you get."_**  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

Semantic Kernel can maintain state with the ChatHistory object. This allows natural converation between the user and AI to be persisted for future use. It can be saved for future reference, it can be replayed, it can be forked in certain spots to try different scenarios, it can be used as a starting point for other conversations etc. 

In the scenario below, let's build a multi-turn conversation about a decision over a purchase for a fitness tracker. Notice that each turn the appropriate Chat History roles are populated with the appropriate prompts and assistant (AI) responses. This in effect acts as the Gathering Intelligence step of the Decision Intelligence framework as the conversation information has been persisted and can be re-used as context for the decision scenario.

In [7]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptDecisionConversation = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// User prompt to start a decision conversation about buying a fitness tracker
var userPromptDecisionConversationTurn1 = """
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 
""";

var assistantResponseDecisionConversationTurn1 = """
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.
""";

var userPromptDecisionConversationTurn2 = """
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?
""";

var assistantResponseDecisionConversationTurn2 = """
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.
""";

var userPromptDecisionConversationTurn3 = """
Are there any popular models you would recommend for that balance of battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn3 = """
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.
""";

var userPromptDecisionConversationTurn4 = """
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn4 = """
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 
""";

// Create a new chat history with a multi-turn conversation about buying a fitness tracker
// Notice how the conversation flow uses different roles (user and assistant) to simulate a realistic decision-making dialogue
var chatHistoryDecisionConversation = new ChatHistory();
chatHistoryDecisionConversation.AddSystemMessage(systemPromptDecisionConversation);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn1);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn1);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn2);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn2);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn3);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn3);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn4);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn4);


Using the above conversation history as the "Gathered Intelligence" betweeen the user and the AI assistant, let's make a final decision and ask for final feedback and a decision evaluation. This "Gathered Intelligence" for the decision can be persisted, re-loaded, simulated multiple times, applied to different AI systems to optimize decisions further.  

In [8]:
var userPromptDecisionConversationFinalDecision = """
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts or insights on my decision? 
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationFinalDecision);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryDecisionConversation))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistoryDecisionConversation.AddAssistantMessage(chatResponse);

Great choice! Here are some quick final insights to support your decision:

**Strengths of the Fitbit Inspire 3:**
- **Battery Life:** Typically lasts up to 10 days, supporting consistent sleep tracking without frequent recharging.
- **Sleep Tracking:** Accurately monitors sleep stages (light, deep, REM), offers sleep scores, and provides actionable insights via the Fitbit app.
- **Comfort & Style:** Lightweight, slim, and unobtrusive—ideal for wearing day and night.
- **App Ecosystem:** Syncs easily with most smartphones and integrates with other health apps (such as Apple Health and Google Fit).
- **Value for Money:** Delivers core features of higher-end models at an affordable price.

**Pro Tips:**
- Set a reminder to charge it while showering or during low-activity times to keep it ready for sleep.
- Regularly check for software updates to benefit from new sleep analysis features.
- Explore the Fitbit community for motivation and tips.

**Final Thought:**  
The Inspire 3 is a reliable companion for tracking both steps and sleep, making it a solid decision based on your stated priorities. Enjoy your new tracker and the insights it’ll bring to your wellness journey!

----
### Step 6 - Inspect & Optimize Gathered Intelligence of Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

> **📝 Note:** In the Decision Intelligence framework, chat history can serve as a form of gathered intelligence. Interactions among users, AI models, processes, and agents provide valuable context for effective decision-making. It’s highly recommended to persist chat history objects (agent threads), especially during decision optimization, to ensure critical information is retained and accessible.  

In [9]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistoryDecisionConversation.Count());

// Change this to a string builder and show as markdown
var stringBuilderChatHistory = new StringBuilder();
foreach (var message in chatHistoryDecisionConversation)
{
    // add a new line for each message
    stringBuilderChatHistory.AppendLine($"**{message.Role.ToString().ToUpper()}**:");
    stringBuilderChatHistory.Append($"{message.Content.Replace("#", string.Empty)}");
    stringBuilderChatHistory.AppendLine("\n");
}

// Display the chat history as Markdown
stringBuilderChatHistory.ToString().DisplayAs("text/markdown");

Number of chat interactions: 11


**SYSTEM**:
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

**USER**:
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 

**ASSISTANT**:
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.

**USER**:
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?

**ASSISTANT**:
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.

**USER**:
Are there any popular models you would recommend for that balance of battery life and sleep tracking?

**ASSISTANT**:
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.

**USER**:
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?

**ASSISTANT**:
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 

**USER**:
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts or insights on my decision? 

**ASSISTANT**:
Great choice! Here are some quick final insights to support your decision:

**Strengths of the Fitbit Inspire 3:**
- **Battery Life:** Typically lasts up to 10 days, supporting consistent sleep tracking without frequent recharging.
- **Sleep Tracking:** Accurately monitors sleep stages (light, deep, REM), offers sleep scores, and provides actionable insights via the Fitbit app.
- **Comfort & Style:** Lightweight, slim, and unobtrusive—ideal for wearing day and night.
- **App Ecosystem:** Syncs easily with most smartphones and integrates with other health apps (such as Apple Health and Google Fit).
- **Value for Money:** Delivers core features of higher-end models at an affordable price.

**Pro Tips:**
- Set a reminder to charge it while showering or during low-activity times to keep it ready for sleep.
- Regularly check for software updates to benefit from new sleep analysis features.
- Explore the Fitbit community for motivation and tips.

**Final Thought:**  
The Inspire 3 is a reliable companion for tracking both steps and sleep, making it a solid decision based on your stated priorities. Enjoy your new tracker and the insights it’ll bring to your wellness journey!



By persisting the Decision Intelligence chats and making them available for inspection, can further help the decision-making process. For example you can load the Chat History summarize it or even have AI recommend optimizations for it based on the conversation. 

> **📝 Note:** The process below is simplified to illustrate the power of Gathering Intelligence with user <--> AI interactions. More advanced best practices will be introduced in further sections. 

In [10]:
var userPromptSummarizeTheDecisionChat = """
Briefly summarize the chat conversation about buying a fitness tracker.
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptSummarizeTheDecisionChat);

var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
chatResponse = chatContent.Content;
chatResponse.DisplayAs("text/markdown");

// Optimize the chat history for any future decision interactions
var userPromptOptimizeTheDecisionChat = """
Based on the chat conversation so far...
Provide some recommendations on how the decision interactions could be optimized for decision-making.
What are some considerations for the AI to help the use make even better decisions?
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptOptimizeTheDecisionChat);

var chatContentOptimization = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
var chatResponseOptimization = chatContentOptimization.Content;
chatResponseOptimization.DisplayAs("text/markdown");

Certainly! Here’s a brief summary of your chat conversation:

You are considering buying a fitness tracker and wanted guidance on important features. You identified battery life and sleep tracking as your top priorities. We discussed that typical trackers offer battery life ranging from 2 days to 2 weeks, with many modern models providing advanced sleep insights. Popular models balancing these needs include Fitbit Inspire 3, Fitbit Charge 5, Garmin Vivosmart 5, and Xiaomi Mi Band 7. You also asked about the Apple Watch, which has good sleep tracking but shorter battery life, requiring daily charging. Based on the information, you decided the Fitbit Inspire 3 best fits your needs and budget, and received final tips for making the most of your new tracker.

Certainly! Here’s a summary of the chat conversation and recommendations for enhancing the decision-making process:

### Chat Conversation Summary
- **User’s Need:** Interested in buying a fitness tracker; uncertain which features matter most.
- **Exploration:** User expressed priorities—good battery life and accurate sleep tracking.
- **Information Provided:** Battery life and sleep tracking capabilities of trackers explained, including typical ranges and feature quality.
- **Recommendations:** Several models suggested (Fitbit Inspire 3, Charge 5, Garmin Vivosmart 5, Xiaomi Mi Band 7) that balance battery life and sleep tracking.
- **Comparison:** Brief comparison with the Apple Watch, detailing pros and cons.
- **Decision:** User leaned towards Fitbit Inspire 3. Final advice and tips were provided to support their choice.

---

### Recommendations to Optimize Decision Interactions

**1. Clarifying Needs and Priorities**
- **Prompt the user to rank or weigh their priorities** (e.g., battery life vs. advanced features vs. budget vs. design).
- **Suggest trade-off analyses** early in the conversation, helping users consider what they’re willing to compromise on.

**2. Structured Options Comparison**
- **Provide a comparison matrix** or table highlighting key models versus prioritized features, making differences and similarities visually clear.
- **Include pros, cons, and “best for” scenarios** for each option.

**3. Contextual & Personalized Guidance**
- **Ask follow-up questions about lifestyle and use cases** (e.g., do they want to wear the tracker for swimming? Are they interested in stress tracking or breathing exercises?).
- **Consider integration or compatibility concerns** (other devices/apps they own, ecosystem preferences).

**4. Transparency and Uncertainty**
- **Highlight feature limitations** (e.g., accuracy differences in sleep tracking by brand, battery life variability with settings).
- **Acknowledge information gaps or uncertainties** (e.g., changes due to software updates).

**5. Enabling Next Steps**
- **Offer sources for in-depth reviews or user testimonials.**
- **Suggest “try before you buy” options** or return policies if available.
- **Encourage ongoing evaluation** after purchase (e.g., revisit needs if they change).

---

### AI Considerations for Supporting Better Decisions

- **Proactively gather relevant context** (the user’s routine, other health goals, technical comfort level).
- **Provide dynamic guidance** that adapts as the user’s stated preferences evolve during the discussion.
- **Present a systems view**, showing how the tracker fits into the broader health/wellness or tech ecosystem.
- **Encourage reflection** by summarizing trade-offs after each decision point.
- **Support unbiased and evidence-based recommendations**, flagging any high-marketing claims vs. proven features.

---

By applying these recommendations, decision interactions could be even more tailored, thorough, and empowering for users looking to make informed technology purchases.